In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [52]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
import kaggle

kaggle.api.authenticate()

kaggle.api.dataset_download_files('julian3833/jigsaw-toxic-comment-classification-challenge',path='/content/mydrive/MyDrive',unzip=True)

In [5]:
import numpy as np
import pandas as pd

In [6]:
data = pd.read_csv('/content/mydrive/MyDrive/train.csv');
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [21]:
data = data[['comment_text','toxic']]
data = data[0:1000]

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertForSequenceClassification,BertTokenizer

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = model.to('cuda')

In [11]:
sample_data = ["I am eating apple","I am playing cricket"]
tokenizer(sample_data,padding=True,truncation=True,max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 6207, 102], [101, 1045, 2572, 2652, 4533, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [24]:
X = list(data['comment_text'])
y = list(data['toxic'])
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train,padding=True,truncation=True,max_length=512)
X_val_tokenized = tokenizer(X_val,padding=True,truncation=True,max_length=512)

In [25]:
print(X_train_tokenized['token_type_ids'][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
Train_Dataset = Dataset(X_train_tokenized,y_train)
Test_Dataset = Dataset(X_val_tokenized,y_val)

In [28]:
def computes_metrices(p):
  print(type(p))
  pred,labels =p
  pred = np.argmax(pred,axis=1)

  accuracy = accuracy_score(y_true=labels,y_pred=pred)
  recall = recall_score(y_true=labels,y_pred=pred)
  precision = precision_score(y_true=labels,y_pred=pred)
  f1 = f1_score(y_true=labels,y_pred=pred)

  return {'accuracy':accuracy,'recall':recall,'precision':precision,'f1':f1}


In [40]:

args = TrainingArguments(
    output_dir="/content/mydrive/MyDrive",
    num_train_epochs=10,
    per_device_train_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=Train_Dataset,
    eval_dataset=Test_Dataset,
    compute_metrics=computes_metrices
)


In [41]:
trainer.train()

Step,Training Loss
500,0.061400


Step,Training Loss
500,0.061400
1000,0.000100


TrainOutput(global_step=1000, training_loss=0.03071563518047333, metrics={'train_runtime': 746.8001, 'train_samples_per_second': 10.712, 'train_steps_per_second': 1.339, 'total_flos': 2104888442880000.0, 'train_loss': 0.03071563518047333, 'epoch': 10.0})

In [42]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.3012032210826874,
 'eval_accuracy': 0.97,
 'eval_recall': 0.8095238095238095,
 'eval_precision': 0.8947368421052632,
 'eval_f1': 0.8500000000000001,
 'eval_runtime': 5.8989,
 'eval_samples_per_second': 33.905,
 'eval_steps_per_second': 4.238,
 'epoch': 10.0}

In [46]:
np.set_printoptions(suppress=True)

In [47]:
text = "That was a good point"

inputs = tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')

outputs = model(**inputs)

predications = torch.nn.functional.softmax(outputs.logits,dim=-1)

predictions = predications.cpu().detach().numpy()

predictions

array([[0.99995685, 0.00004316]], dtype=float32)

In [48]:
trainer.save_model('/content/mydrive/MyDrive')

In [49]:
model2 = BertForSequenceClassification.from_pretrained('/content/mydrive/MyDrive')
model2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [53]:
text = "don't mess with me"

inputs = tokenizer(text,padding=True,truncation=True,return_tensors='pt').to('cuda')

outputs = model2(**inputs)


predications = torch.nn.functional.softmax(outputs.logits,dim=-1)

predictions = predications.cpu().detach().numpy()

predictions

array([[0.00004945, 0.9999505 ]], dtype=float32)